# Example of how to access Proba-2 data via python

##### Author: ESDC team at ESAC

This notebook illustrates some examples of how to access P2SA data using the Table Access Protocol (TAP) and the Astronomical Query Language (ADQL) in Python. See [here](http://p2sa.esac.esa.int/p2sa/#aio ) for more information and examples of ADQL queries.

The documentation for sunPy is available [here](https://docs.sunpy.org/en/stable/).

## Examples

### 1. BASIC Use
#### 1.1 Getting public tables¶


This example illustrates how to download only public P2SA table names by using TAP+ capability. 

In [ ]:
# Import TapPlus capability (Not available yet in SunPy)
from astroquery.utils.tap.core import TapPlus

p2sa = TapPlus(url=TAP_URL)
tables = p2sa.load_tables(only_names=True)
for table in (tables):
    print(table.get_qualified_name())

In [ ]:
from astroquery.utils.tap.core import TapPlus

p2sa = TapPlus(url=TAP_URL)
tables = p2sa.load_tables()
for table in (tables):
    print(table.get_qualified_name())

To load only a table (TAP+ capability)


In [ ]:
from astroquery.utils.tap.core import TapPlus

p2sa = TapPlus(url=TAP_URL)
table = p2sa.load_table('p2sa.v_observation');
print(table);


Once a table is loaded, columns can be inspected

In [ ]:
from astroquery.utils.tap.core import TapPlus

p2sa = TapPlus(url=TAP_URL)
table = p2sa.load_table('p2sa.v_observation');
for column in (table.columns):
        print(column.name);

#### 1.2. Synchronous query

A synchronous query will not store the results at server side. These queries must be used when the amount of data to be retrieved is 'small'.

The results can be saved in memory (default) or in a file. 

Here is an example of query without saving results in memory:

In [ ]:
from astroquery.utils.tap.core import TapPlus

p2sa = TapPlus(url=TAP_URL)

job = p2sa.launch_job("select top 100 begin_date, calibrated, \
                       end_date, file_format, file_name, file_path, file_size, \
                       instrument_oid, observation_oid, observation_type, \
                       processing_level, science_objective, science_object_oid, wavelength_range \
                       from p2sa.observation")
print(job)

In [ ]:
r = job.get_results()
print(r['file_name'])

Query saving results in a file:

In [ ]:
from astroquery.utils.tap.core import TapPlus

p2sa = TapPlus(url=TAP_URL)

job = p2sa.launch_job("select top 100 begin_date, calibrated, \
                       end_date, file_format, file_name, file_path, file_size, \
                       instrument_oid, observation_oid, observation_type, \
                       processing_level, science_objective, science_object_oid, wavelength_range \
                       from p2sa.observation", dump_to_file=True)
print(job)

In [ ]:
r = job.get_results()
print(r['file_name'])


#### 1.3. Asynchronous query

Asynchronous queries save results at the server side. These queries can be accessed at any time.
The results can be saved in memory (default) or in a file. Here is an example of query without saving results in memory:

In [ ]:
from astroquery.utils.tap.core import TapPlus

p2sa = TapPlus(url=TAP_URL)

job = p2sa.launch_job_async("select top 100 begin_date, calibrated, \
                       end_date, file_format, file_name, file_path, file_size, \
                       instrument_oid, observation_oid, observation_type, \
                       processing_level, science_objective, science_object_oid, wavelength_range \
                       from p2sa.observation")
print(job)

#### 1.4. Asynchronous job removal

In [ ]:
from astroquery.utils.tap.core import TapPlus

p2sa = TapPlus(url=TAP_URL)

job = p2sa.remove_jobs("1586707834624OPER")


### 2. P2SA Use Cases:

#### 2.1 Quering P2SA Metadata - Basic Search

##### Example 1:

This first example shows an abreviated way to display the P2SA observations available for the given interval and the list of instruments provided. For this example, we will retrieve all the observations made with the instruments "SWAP" and "LYRA" from the 6th to the 8th of May, 2015.
The result will be a file in 'votable' format.

For this example we will use the method __ESAP2SA.query_p2sa_observations()__ which accepts the following parameters:

- __*instruments*__: List of with the name of the instruments selected for the example, "SWAP" and "LYRA". Optional
- __*to_date*__: String with format (yyyy-mm-dd 00:00:00). Optional
- __*from_date*__: String with format (yyyy-mm-dd 00:00:00). Optional
- __*output_format*__: This can be 'csv', 'votable_plain', 'votable'. Default Votable. Optional
- __*filename*__: File name to be used to store the metadata. Default Nonte. Optional.
- __*verbose*__: Boolean. Flag to display information about the process. Default False. Optional

In [ ]:
from esa_p2sa.p2sa_core import ESAP2SA

instrument_list = ["SWAP", "LYRA"]

ESAP2SA.query_p2sa_observations(instruments=instrument_list, to_date="2015-05-08 00:00:00", from_date="2015-05-06 00:00:00", output_format="votable", filename='p2sa_observations_SWAP_LYRA_May_2015.vot')

#RETURN_TYPE can be VOTABLE, CSV or JSON

##### Example 2:

For this method "__query_p2sa_observations()__" is not mandatory to provide values for all the parameters. An empty parameter just means that this value will not be take it into account for the query. 

In the following example we are going to retrieve observations made with the instrument "SWAP" since the 1st of June, 2017 until now. 

The result will be a file in 'votable' format.

In [ ]:
from esa_p2sa.p2sa_core import ESAP2SA

instrument_list = ["SWAP"]

ESAP2SA.query_p2sa_observations(instruments=instrument_list, to_date="", from_date="2017-06-01 00:00:00",output_format="votable", filename='p2sa_observations_swap.vot')

#RETURN_TYPE can be VOTABLE, CSV or JSON

##### 2.2 Quering P2SA Metadata - Advanced Search

The example belows show a simple ADQL request to get the p2sa observations made from the 7th of January, 2017 and the 15th of January, 2017 with the instrument 'SWAP'. The result metadata will be download into CSV file called 'metadata.csv'.


For this example we will use the method __ESAP2SA.query_p2sa_tap()__ which accepts the following parameters:

- __*query*__: String with the request in ADQL format. Mandatory.
- __*output_format*__: This can be 'csv', 'votable_plain', 'votable'. Default Votable. Optional
- __*output_file*__: File name to be used to store the metadata. Default Nonte. Optional.
- __*verbose*__: Boolean. Flag to display information about the process. Default False. Optional
- __*dump_to_file*__: Boolean. Save results in a file. By default is False. Optional

In [ ]:
from esa_p2sa.p2sa_core import ESAP2SA

ESAP2SA.query_p2sa_tap(query="select * from p2sa.v_observation obs where ((obs.end_date > '2017-01-07 00:00:00') and (obs.begin_date < '2017-01-15 00:00:00') and (obs.instrument_name = 'SWAP'))",output_file="p2sa_metadata_query.vot",output_format="votable",verbose="False", dump_to_file=True)

##### 2.3 Download P2SA data

###### 2.3.1 Download Postcards

The following example illustrates how to download a specific postcard from P2SA.
The __*ESAP2SA.get_p2sa_postcard()*__ function provides access to the P2SA database using the Table Access Protocol (TAP). This method accepts the following paramenters:

- __*observation_oid*__: String with the observation oid number. This number can be obtained as a result of a metadata request. Mandatory.
- __*resolution*__: String. Optional
- __*verbose*__: Boolean. Flag to display information about the process. Default False. Optional

By default, this method downloads the postcard to the current directory but can be displayed by executing the following line "Image(url=link)"

Here is an example:

In [ ]:
from esa_p2sa.p2sa_core import ESAP2SA

from IPython.display import Image
link = ESAP2SA.get_p2sa_postcard(observation_oid="202592722",resolution="low",verbose="True")

# Display the result postcard.
Image(url=link)

###### 2.3.2 Display Carrington movie

The __*ESAP2SA.get_p2sa_movie()*__ function allows to display SWAP daily movie files as well as SWAP Carrignton movie files. This method needs a *file_oid* as input parameter to download and display the movie. This *file_oid* value can be obtained as a result of a metadata request to P2SA tap. (See method *ESAP2SA.query_p2sa_tap()* in Section 2.2)

Here is an example:

In [ ]:
from esa_p2sa.p2sa_core import ESAP2SA
from IPython.display import Video
link = ESAP2SA.display_p2sa_movie_file(file_oid="13785")

# Display the result Carrington movie.
Video(link)

###### 2.3.3 Download P2SA product

The following example illustrates how to download a specific product from P2SA. The __*ESAP2SA.get_p2sa_product()*__ method provides access to the P2SA database using the Table Access Protocol (TAP) and downloads the file required as parameter.

This method accepts as parameter a list of strings with the oids of the files to download. Once the request is completed, the files will be downloaded by default to the directory in which this Jupyter Notebook is being executed.

The following is an example of use of this method:

In [ ]:
from esa_p2sa.p2sa_core import ESAP2SA
import numpy as np

#As example create a list with the files we want to download

file_oid_list = np.array(['6605', '6606'])
print(file_oid_list)

ESAP2SA.get_p2sa_product(file_oid_list=file_oid_list)